# Start 

We begin with a leave-one-out participant to get for every participant a number of wrong selections and how many it would be with the help of TCNAE.

In [1]:
# save folder
import os
import utils

save_folder = utils.create_unique_folder('Results/PowerMeasurements')

In [ ]:
import utils
from TCN import *
import torch
import numpy as np
import json
import train

model_folder = "Results/LOSO_a200_b300_f44_seq43/Models"
model_base_name = "TCNAE_1"
conditions = ["gaze", "headAndGaze", "nod"]

fps = 90
f = 44
b = 300
a = 200

for cond in conditions:
    print(f"Condition: {cond}")
    model_name = f"{model_base_name}_{cond}"
    with open(os.path.join(model_folder, model_name + "_info.json"), 'r') as f:
        model_info = json.load(f)

    angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}_finalRounds.npy")
    angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}_finalRounds.npy")
    names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}_finalRounds.npy")
    names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}_finalRounds.npy")

    
    pat_names = np.unique(names_correct)

    for k, name in enumerate(pat_names):
        train_fold = angles_correct[names_correct != name]
        test_fold_correct = angles_correct[names_correct == name]
        test_fold_incorrect = angles_incorrect[names_incorrect == name]

        model = TCNAE(**model_info["model_params"])
        model, _ = train.train_autoencoder(
            model=model,
            train_data=train_fold,
            batch_size=model_info["train_parameter"]["batch_size"],
            num_epochs=model_info["train_parameter"]["num_epochs"],
            learning_rate=model_info["train_parameter"]["learning_rate"],
            criterion=torch.nn.MSELoss(),
            use_gpu=model_info["train_parameter"]["use_gpu"],
            desc_tqdm=f"Participant {k+1}/{len(pat_names)} {name}"
        )

        mse_train, mse_correct, mse_incorrect = train.test_autoencoder(
            model=model,
            train_data=train_fold,
            test_data_correct=test_fold_correct,
            test_data_incorrect=test_fold_incorrect,
            use_gpu=model_info["train_parameter"]["use_gpu"],
            batch_size=model_info["train_parameter"]["batch_size"]
        )

        t = np.percentile(mse_train.cpu().numpy(), model_info["th_perc"])

        results = {
            "participant": name,
            "mse_correct": mse_correct.cpu().numpy().tolist(),
            "mse_incorrect": mse_incorrect.cpu().numpy().tolist(),
            "threshold": t
        }
        with open(os.path.join(save_folder, f"Results_{name}.json"), 'w') as f:
            json.dump(results, f)

    

Condition: gaze


FileNotFoundError: [Errno 2] No such file or directory: "Data/Dataset_Prepare/angles_fps90_gaze_Correct_f<_io.TextIOWrapper name='Results/LOSO_a200_b300_f44_seq43/Models\\\\TCNAE_1_gaze_info.json' mode='r' encoding='utf-8'>_b300_a200_finalRounds.npy"